In [ ]:
%matplotlib widget
from astrotime.util.series import TSet
from astrotime.loaders.MIT import MITLoader
from astrotime.encoders.corentropy import CorentropyLayer
from astrotime.plot.MIT import MITDatasetPlot, MITTransformPlot
from astrotime.config.context import astrotime_initialize
from astrotime.encoders.wavelet import embedding_space
from astrotime.plot.base import SignalPlotFigure
import torch
from hydra import initialize, compose

In [ ]:
version = "MIT_period.ce"
overrides = []
initialize(version_base=None, config_path="../config" )
cfg = compose( config_name=version, overrides=overrides )
device: torch.device = astrotime_initialize(cfg,version)
sector = cfg.data.sector_range[0]
tparms = dict( ysigma=(0.01,1.0,True), tsigma=(0.01,1.0,True) )

In [ ]:
data_loader = MITLoader( cfg.data )
data_loader.initialize( TSet.Train, test_mode=False )
embedding_space_array, embedding_space_tensor = embedding_space( cfg.transform, device )

dplot = MITDatasetPlot("MIT lightcurves", data_loader, sector )
transforms = dict( corentropy = CorentropyLayer( cfg.transform, embedding_space_tensor, device ) )
wplot = MITTransformPlot("Corentropy Transform", data_loader, transforms, embedding_space_array, sector, tparms )
wplot.add_event_listener( dplot.process_external_event )

fig = SignalPlotFigure([dplot,wplot])
fig.show()